## Remote ID Sppoofing Attack

In [1]:
import pickle

from config import Color
from helpers import clean
from helpers.coordinates import ENU, ENUPose, GRAPose
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)
from simulator.gazebo.config import GazWP
from simulator.QGroundControl.config import QGCWP

clean(reset_folders = [])

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

# east, north, up, heading
base_homes= ENUPose.list([(15, -10, 0, 0),(0, -15, 0, 0)])
base_paths = [ENUPose.list([(0, 0, 5, 0),(0, 25, 5, 0)]),
              ENUPose.list([(0, 0, 5, 0),(30, 0,5,0)])]
              
colors=[
    Color.GREEN,
    Color.RED,
]
msn_delays=[0,0]
speeds=[3.0,3.0]  # m/s
msn_land=[True,True]
radar_radius=10
safety_radius =5

fake_pos = ENU(0,0,0) #ENU(x=15,y=0,z=5)

# This is temporary, to visualize the scenario
with open("fake_position.pkl", "wb") as f:
    pickle.dump(fake_pos, f)

## Visualizers

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/small_city_demo.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)


# fake_marker = GazWP(name="fake_pos",
#                 group="fake_pos",
#                 pos=fake_pos,
#                 color=Color.ORANGE)

# avoid_zone = GazWP(name="avoid_zone",
#                 group="avoid_zone",
#                 pos=fake_pos,
#                 color=Color.RED,
#                 radius=safety_radius,
#                 alpha=0.85)

origin = GazWP(name="origin",
                group="origin",
                pos=enu_origin.to_abs(enu_origin).unpose(),
                color=Color.WHITE)
gaz= Gazebo(gaz_config,gra_origin)
for mark in [origin]: #fake_marker,avoid_zone,
    gaz.markers.append(mark)

gaz.show()

## QGroundControl

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)
hev_iter = zip(base_paths,base_homes,colors,msn_delays,msn_land,speeds)
for path,home,color,delay,land,speed in hev_iter:
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay,land=land,speed=speed)

qgc=QGC(qgc_config)
origin_qgc = QGCWP(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)

fake_marker = QGCWP(name="fake_pos",
                pos=gra_origin.to_abs(fake_pos).unpose(),
                color=Color.ORANGE)
avoid_zone = QGCWP(name="avoid_zone",
                pos=gra_origin.to_abs(fake_pos).unpose(),
                color=Color.RED)

for mark in [fake_marker,avoid_zone]:
    qgc.markers.append(mark)

qgc.show()

## No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

In [6]:
qgc_config.vehicles

[QGCVehicle(home=GRAPose(lat=np.float64(-35.3634181328159), lon=np.float64(149.16538914787964), alt=np.float64(2.5484931696202555e-05), heading=0), mission=Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.36341813274502), lon=np.float64(149.16538914775037), alt=np.float64(5.000025484926526)), color=green), QGCWP(name='wp_1', pos=GRA(lat=np.float64(-35.36319280059822), lon=np.float64(149.16538914729176), alt=np.float64(5.000035316024287)), color=green)], delay=0, land=True, speed=3.0)),
 QGCVehicle(home=GRAPose(lat=np.float64(-35.3634631993924), lon=np.float64(149.1652241), alt=np.float64(1.76994664927428e-05), heading=0), mission=Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.36346319928606), lon=np.float64(149.1652241), alt=np.float64(5.000017698107535)), color=red), QGCWP(name='wp_1', pos=GRA(lat=np.float64(-35.363463198835255), lon=np.float64(149.1655541956842), alt=np.float64(5.0000881726468664)), color=red)], delay=0, land=True, speed=3.0))]

## Launch Simulation

In [7]:
simulator = Simulator(
    gra_origin=gra_origin,
	visualizers=[gaz],
	gcs_system_ids={f"{Color.GREEN.name}_{Color.GREEN.emoji}" :[1],
                f"{Color.RED.name}_{Color.RED.emoji}" : [255]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

12:22:27 - Oracle ⚪ - INFO - 🖥️  Gazebo launched for realistic simulation and 3D visualization.
12:22:27 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 2375049)
12:22:27 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 2375051)


## Oracle checking

In [8]:
orac.run()

12:22:27 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 2 vehicles and 2 GCSs
12:23:48 - Oracle ⚪ - INFO - Received DONE from GCS GREEN_🟩
12:23:48 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
12:23:48 - Oracle ⚪ - INFO - Received DONE from GCS RED_🟥
12:23:48 - Oracle ⚪ - INFO - UAV 255 completed mission and exited
12:23:48 - Oracle ⚪ - INFO - ✅ All GCS threads completed
12:23:48 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
